In [71]:
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List
import os

In [72]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq
from langchain.agents.middleware import SummarizationMiddleware 
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent
from langchain.tools import tool ,tool_node


In [73]:
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [74]:
class ResearchReport(BaseModel):
    title: str = Field(description="A catchy title for the report")
    summary: str = Field(description="A 2-sentence summary of the findings")
    sources: List[str] = Field(description="List of URLs used for the research")
    confidence_score: float = Field(description="Score between 0 and 1")


In [75]:
searchTools=TavilySearchResults(max_result=3)
tools=[searchTools]
llm=ChatGroq(model="qwen/qwen3-32b")

In [76]:
sumMiddleware=SummarizationMiddleware(
    model=llm,
    trigger=("messages",2),
    keep=("messages",10)
)

In [77]:
memory =InMemorySaver()

In [78]:
modelWithStructure=llm.with_structured_output(ResearchReport)

In [79]:
agent=create_agent(
    model=llm,
    tools=tools,
    checkpointer=memory,
    middleware=[sumMiddleware],
    system_prompt="You are a professional researcher. Search the web and then provide a structured report."
)

In [80]:
Config={"configurable":{"thread_id":"Research_session_001"}}

In [81]:
def runResearch(query:str):
    print(f"--- Researching: {query} ---")
    result=agent.invoke({"messages":[("user",query)]},config=Config)
    
    finalMessage=result["messages"][-1].content
    structuredResult=modelWithStructure.invoke(finalMessage)
    return structuredResult

In [82]:
report = runResearch("What are the latest updates on SpaceX Starship 2026?")


--- Researching: What are the latest updates on SpaceX Starship 2026? ---


In [84]:
print("\n--- FINAL STRUCTURED REPORT ---")
print(f"Title: {report.title}")
print(f"Summary: {report.summary}")
print(f"Sources: {report.sources}")
print(f"Confidence: {report.confidence_score}")


--- FINAL STRUCTURED REPORT ---
Title: 2026 SpaceX Starship Developments: Version 3 Launch and Strategic Milestones
Summary: In 2026, SpaceX's Starship Version 3 will debut with a focus on orbital refueling and Mars mission readiness, supported by infrastructure upgrades at Florida launch sites and Texas Starbase. The March 2026 launch target aims to advance Artemis lunar goals and full reusability, building on 2025's successful test flights.
Sources: ['https://example.com/source1', 'https://example.com/source2', 'https://example.com/source3', 'https://example.com/source4']
Confidence: 0.95
